# 설치
  - `pip install langchain langchain-qdrant langchain-openai python-dotenv`

# Vector Store 생성
- QDrantClient로 Database에 연결
    - Collection 생성(없으면)
- VectorSore 생성
    - client와 collection name, embedding 모델을 전달해서 생성

In [ ]:
from uuid import uuid4
from langchain_core.documents import Document
data = [
    {
        "name": "The Time Machine",
        "description": "시간 여행자는 미래로 여행을 떠나 인류의 진화와 몰락을 목격하게 된다. 그는 멀리 떨어진 미래 사회에서 인간 문명이 퇴화한 모습을 보고 충격을 받는다. 이 이야기는 인간 문명과 과학기술의 진보가 가져올 결과를 철학적으로 탐구한다.",
        "author": "H.G. Wells",
        "year": 1895
    },
    {
        "name": "Ender's Game",
        "description": "소년 엔더는 외계 종족 '버거'와의 전쟁을 대비해 혹독한 군사 훈련을 받는다. 그는 전략적 천재성을 발휘하여 점차 전쟁의 중심에 서게 된다. 이야기는 어린아이를 도구로 삼는 전쟁의 비인간성과 심리적 고통을 그린다.",
        "author": "Orson Scott Card",
        "year": 1985
    },
    {
        "name": "Brave New World",
        "description": "유전자 조작과 세뇌로 계급이 철저히 나뉘어진 미래 사회가 배경이다. 인간은 자유 없이 쾌락과 안정을 주입받으며 살고, 비순응자는 배제된다. 디스토피아 속에서 인간 본성과 자유의 의미를 묻는 소설이다.",
        "author": "Aldous Huxley",
        "year": 1932
    },
    {
        "name": "The Hitchhiker's Guide to the Galaxy",
        "description": "지구 멸망 직후 우주로 탈출한 주인공이 외계 친구와 함께 기상천외한 모험을 시작한다. 우주의 황당한 존재들과 철학적 농담들이 넘치는 코믹 SF다. 우주에 대한 진지한 질문을 유쾌하게 풀어낸 작품이다.",
        "author": "Douglas Adams",
        "year": 1979
    },
    {
        "name": "Dune",
        "description": "사막 행성 아라키스를 둘러싼 정치적 음모와 권력 투쟁이 벌어진다. 주인공 폴은 운명과 예언에 따라 거대한 반란을 이끌게 된다. 정치, 종교, 생태학이 얽힌 거대한 서사의 서막이다.",
        "author": "Frank Herbert",
        "year": 1965
    },
    {
        "name": "Foundation",
        "description": "수학자 하리 셀던은 미래를 예측하는 '심리역사학'을 개발해 인류의 몰락을 막기 위한 계획을 세운다. 그는 제국의 붕괴 속에서도 문명을 재건할 수 있는 재단을 설립한다. 과학과 지식의 힘이 문명을 구원할 수 있는지를 탐구한다.",
        "author": "Isaac Asimov",
        "year": 1951
    },
    {
        "name": "Snow Crash",
        "description": "가상현실 '메타버스'와 현실이 뒤얽힌 미래에서, 주인공 해커는 '스노 크래시'라는 치명적인 바이러스를 추적한다. 정보와 언어, 권력이 하나로 연결된 세계가 그려진다. 포스트사이버펑크의 대표작으로 인터넷 이후의 세계를 예견한다.",
        "author": "Neal Stephenson",
        "year": 1992
    },
    {
        "name": "Neuromancer",
        "description": "해커 '케이스'는 거대 기업의 의뢰를 받아 사이버 공간에서 불가능한 해킹을 수행한다. 임무를 수행하며 인공지능과 인간의 경계가 모호해지는 음모에 휘말린다. 사이버펑크 장르의 원형으로 가상 현실과 인간 의식의 융합을 다룬다.",
        "author": "William Gibson",
        "year": 1984
    },
    {
        "name": "The War of the Worlds",
        "description": "화성인이 지구를 침략하면서 인류는 공포와 절망 속에 빠진다. 압도적인 기술력 앞에 속수무책으로 무너지는 인간의 모습이 그려진다. 외계 생명체와 문명의 충돌을 통해 인간 중심적 세계관에 경종을 울린다.",
        "author": "H.G. Wells",
        "year": 1898
    },
    {
        "name": "The Hunger Games",
        "description": "디스토피아 사회에서 어린이들은 생존 게임에 강제로 참가해야 한다. 주인공 캣니스는 잔혹한 게임 속에서 인간성과 저항의 상징이 되어간다. 권력과 억압, 생존 본능의 이야기가 펼쳐진다.",
        "author": "Suzanne Collins",
        "year": 2008
    },
    {
        "name": "The Andromeda Strain",
        "description": "우주에서 온 미생물이 미국의 한 마을을 초토화시키자, 과학자들이 이를 분석하고 대응에 나선다. 시간과의 싸움 속에서 인류의 과학 기술과 오만함이 드러난다. 생물무기와 전염병의 공포를 현실감 있게 묘사한 작품이다.",
        "author": "Michael Crichton",
        "year": 1969
    },
    {
        "name": "The Left Hand of Darkness",
        "description": "외교관 제너리 아이가 젠더가 유동적인 외계 종족이 사는 행성에 파견된다. 그는 그들과의 문화적 충돌과 이해를 통해 진정한 인간성과 연대를 배워간다. 젠더와 정체성, 타자성에 대한 철학적 탐구가 돋보이는 작품이다.",
        "author": "Ursula K. Le Guin",
        "year": 1969
    },
    {
        "name": "The Three-Body Problem",
        "description": "문화대혁명기의 중국에서 외계 문명과의 접촉이 시작되며 인류는 큰 위기에 직면한다. 세 개의 태양이 있는 불안정한 행성에 사는 외계인과의 접촉이 인류 문명에 위협을 가한다. 과학, 철학, 문명 충돌을 고찰하는 하드 SF 걸작이다.",
        "author": "Liu Cixin",
        "year": 2008
    }
]

documents = [
    Document(page_content=doc['description'], metadata=doc, id=str(uuid4())) for doc in data
]
documents

# 검색

# Qdrant 세가지 방식

- **Dense Vector Search** (default): **의미 기반** 검색. Vector간 의미적 유사도록 검색
- **Sparse Vector Search**: **키워드 기반** 검색.
- **Hybrid Search**: 두가지 방법을 합쳐서 검색

## Dense Vector Search
- Default 방식
- `RetrievalMode.DENSE` 로 설정
- embedding 파라미터에 값이 제공되야 한다.

### Sparse Vector Search
- **희소 표현(sparse representation)을** 사용한 벡터 검색 방식.
- 이 기능은 특히 BM25, TF-IDF, Bag-of-Words 검색과 같은 전통적인 정보 검색(IR: Information Retrieval) 기법을 이용해 검색을 한다.
- 보통 이 방식을 단독으로 쓰지 않고 의미기반 검색을 보완하기 위해 사용한다. (Hybrid 검색)
- `pip install -qU fastembed`
  
#### 설정
- sparse vector search 만 할 경우
  - **collection 생성** 시 `sparse_vectors_config` 에 sparse vector 설정을 한다.
  - Langchain VectorStore 생성시 `sparse_embedding_model`을 설정하고 collection 생성시 지정한 설정을 `sparse_vector_name` 파라미터를 이용해 전달한다.

#### BM25 (Best Matching 25) 알고리즘

BM25는 검색 엔진이나 추천 시스템에서 **사용자가 입력한 검색어와 가장 관련성이 높은 문서**를 찾아 순위를 매기는 데 사용되는 알고리즘이다.   
통계 기반의 정보 검색 기법 중 하나로, 특히 문서 간의 관련성을 정량적으로 평가할 때 많이 사용된다.

BM25는 다음 세 가지 주요 요소를 바탕으로 점수를 계산한다:


##### 1. **단어의 출현 빈도 (Term Frequency, TF)**

- 문서 내 특정 단어가 얼마나 자주 등장하는지를 측정한다.
- 사용자가 검색한 단어가 어떤 문서에서 많이 등장할수록, 해당 문서는 검색어와 관련성이 높다고 판단한다.
- 하지만 너무 자주 등장하는 단어에 대해서는 **점수 증가를 둔화**시키는 방식(로그나 분수 기반의 함수)을 사용한다. 즉, 단어의 빈도 수가 많다고 무한정 점수가 올라가지는 않는다.

##### 2. **단어의 희소성 (Inverse Document Frequency, IDF)**

- 전체 문서(검색 대상 문서) 집합에서 특정 단어가 얼마나 희귀한지를 평가한다.
- 많은 문서에 동시에 등장하는 단어는 정보성이 낮기 때문에 낮은 점수를 부여하고, 드물게 등장하는 단어에는 높은 점수를 부여한다.

##### 3. **문서의 길이 보정 (Document Length Normalization)**

- 문서의 길이에 따라 점수를 보정한다.
- 같은 단어가 등장한 횟수가 같더라도, 짧은 문서에서는 그 단어의 중요도가 더 높다고 판단한다.
- 너무 긴 문서가 단지 길이 때문에 더 높은 점수를 받는 것을 방지하기 위해, 평균 문서 길이를 기준으로 보정한다.

### BM25 점수 계산 과정

1. **검색어 토큰화**
   - 사용자가 입력한 검색어 문장을 단어(토큰) 단위로 분리한다.
   - 예: "강아지 산책 방법" → \["강아지", "산책", "방법"\]
2. **각 문서에 대해 단어별 점수 계산**
   - 분리된 각 단어에 대해, 위 세 가지 요소를 기반으로 각 대상 문서의 점수를 계산한다:
3. **문서별 총점 산출**
   - 각 단어별 점수를 모두 더해 해당 문서의 총점을 계산한다.
4. **문서 순위화**
   - 점수가 높은 문서일수록 검색어와 관련성이 높다고 판단하여, 그 순서대로 결과를 정렬한다.



### BM25의 한계

- **의미 파악 불가**: BM25는 단어의 단순 일치 여부만 평가하며, 단어의 의미나 문맥은 고려하지 않는다. 예를 들어 "사과"가 "과일"인지 "용서"인지 구분하지 못한다.
- **동의어 처리 어려움**: "자동차"와 "승용차"처럼 다른 단어지만 유사한 의미를 가진 경우, BM25는 이들을 동일하게 인식하지 못한다.
- **자연어 이해 부족**: 문장의 구조나 어순, 구문 정보는 활용하지 않는다.
- **언어별 특성**- 한국어처럼 교착어에서는 형태소 분석이 중요한데 이에 대한 고려가 부족
- **구문 관계**: 단어 간의 거리나 위치 정보를 고려하지 않음

BM25는 간단하면서도 강력한 정보 검색 알고리즘으로, **키워드 기반 검색**에 매우 효과적이다. 다만 단어의 **의미나 문맥을 이해하지 못한다는 점**에서 **의미 기반 검색 모델**이 보완적으로 사용되기도 한다.


### Hybrid
- Hybrid Search는 Dense Vector Search와 Sparse Vector Search를 결합하여, **의미 기반 검색(semantic search)**과 **정확한 키워드 검색(lexical search)**의 장점을 모두 활용하는 검색 방식이다.
- RAG(Retrieval-Augmented Generation) 시스템이나 검색 기반 질문응답 시스템에서 정확도와 재현율을 동시에 향상시키는 데 매우 유용하다.

> - Dense Vector: 문장의 의미를 파악하는 임베딩 기반 검색
> - Sparse Vector: 키워드 기반의 전통 정보 검색(BM25, TF-IDF 등)

- Dense vector만 사용하면 문맥적 의미는 이해하지만 키워드가 누락된 문서가 나올 수있다.
- Sparse vector만 사용하면 정확한 단어, 키워드는 찾지만 의미 파악이 부족할 수있다.
- 위 두가지 방식의 단점을 보완한 것이 Hybrid 방식으로 **정확한 키워드와 의미적으로 유사한 문서**를 함께 찾는다.
- 하이브리드 방식을 적용하면
  - BM25로 1차 후보 검색
  - Dense Embedding으로 의미 기반 재정렬